Necessary imports

In [1]:
import sys
sys.path.append('../../')

from pyspark.sql import SparkSession
from pyspark.ml import Pipeline

from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.base import *

from pathlib import Path

if sys.version_info[0] < 3:
    from urllib import urlretrieve
else:
    from urllib.request import urlretrieve


Create SparkSession if not already in pyspark

In [2]:
spark = SparkSession.builder \
    .appName("assertion-status")\
    .master("local[2]")\
    .config("spark.driver.memory","4G")\
    .config("spark.driver.maxResultSize", "2G")\
    .config("spark.jar", "lib/sparknlp.jar")\
    .config("spark.kryoserializer.buffer.max", "500m")\
    .getOrCreate()

In [3]:
import time


embeddingsFile = 'PubMed-shuffle-win-2.bin'
embeddingsUrl = 'https://s3.amazonaws.com/auxdata.johnsnowlabs.com/PubMed-shuffle-win-2.bin'
# this may take a couple minutes
if not Path(embeddingsFile).is_file():
    urlretrieve(embeddingsUrl, embeddingsFile)

documentAssembler = DocumentAssembler()\
    .setInputCol("sentence")\
    .setOutputCol("document")\

assertion = AssertionLogRegApproach()\
    .setLabelCol("label")\
    .setInputCols(["document"])\
    .setOutputCol("assertion")\
    .setBefore(11)\
    .setAfter(13)\
    .setEmbeddingsSource(embeddingsFile,200,3)


finisher = Finisher() \
    .setInputCols(["assertion"]) \
    .setIncludeKeys(True)

pipeline = Pipeline(
    stages = [
    documentAssembler,
    assertion,
    finisher
  ])


We load the parquet dataset and put it in memory

In [4]:
#Load the input data to be annotated
data = spark. \
        read. \
        parquet("../../../src/test/resources/negex.parquet"). \
        limit(3000)
data.cache()
data.count()
data.show()

+--------------------+--------------------+--------+-----+---+
|            sentence|              target|   label|start|end|
+--------------------+--------------------+--------+-----+---+
|**initials ______...|multinodular goit...|Affirmed|   21| 25|
|02) mild aortic r...|mild aortic regur...|Affirmed|    1|  3|
|02) mild left atr...|mild left atrial ...|Affirmed|    1|  4|
|02) mild left atr...|mild left atrial ...|Affirmed|    1|  4|
|02) mild to moder...|mild to moderate ...|Affirmed|    1|  5|
|02) mild to moder...|mild to moderate ...|Affirmed|    1|  5|
|02) no valvular a...|valvular abnormal...| Negated|    2|  3|
|02) nondilated ri...|nondilated right ...|Affirmed|    1|  9|
|02) normal left v...|normal left ventr...|Affirmed|    1|  4|
|02) normal left v...|normal left ventr...|Affirmed|    1|  6|
|02) paradoxical s...|post-operative se...|Affirmed|    6|  8|
|02) small left ve...|small left ventri...|Affirmed|    1|  8|
|03) mild mitral r...|mild mitral regur...|Affirmed|   

We fit the data

In [5]:
start = time.time()
print("Start fitting")
model = pipeline.fit(data)
print("Fitting is ended")
print (time.time() - start)

Start fitting
Fitting is ended
136.5720419883728


Prediction with finisher output

In [ ]:
result = model.transform(data)
result.select("sentence", "target", "finished_assertion").show()

Save the model to disk and read it back

In [ ]:
pipeline.write().overwrite().save("./assertion_pipeline")
model.write().overwrite().save("./assertion_model")

In [ ]:
from pyspark.ml import PipelineModel, Pipeline

Pipeline.read().load("./assertion_pipeline")
sameModel = PipelineModel.read().load("./assertion_model")

In [ ]:
sameModel.transform(data).select("sentence", "target", "finished_assertion").show()

Load a pretrained model for assertion status

In [ ]:
pretrained = AssertionLogRegModel.pretrained()